In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

# 读写文件

具体可以参考下面API调用说明：

> https://pytorch.org/cppdocs/api/function_namespacetorch_1ac9e56fddb5d7312193bb49ca525ec55e.html?highlight=torch%20save
> https://pytorch.org/cppdocs/api/function_namespacetorch_1a4b369494adfb10b9a005aeb0bb6207cb.html?highlight=torch%20load

<img style="float: left;" src="./img/save1.png" width="75%">

<img style="float: left;" src="./img/save2.png" width="75%">

<img style="float: left;" src="./img/load1.png" width="75%">

<img style="float: left;" src="./img/load2.png" width="75%">

**加载和保存张量**

In [3]:
auto x = torch::arange(0, 4);
torch::save(x, "x.tensor");

torch::Tensor x2; 
torch::load(x2, "x.tensor");
printT(x2);

x2 = 
 0
 1
 2
 3
[ CPULongType{4} ]
<<--->>



**存储一个张量列表，然后把它们读回内存**

In [4]:
auto y = torch::zeros(4);
torch::save({x, y}, "x1.tensor");

In [5]:
std::vector<torch::Tensor> tv;
torch::load(tv, "x1.tensor");
printT(tv[0]);
printT(tv[1]);

tv[0] = 
 0
 1
 2
 3
[ CPULongType{4} ]
<<--->>

tv[1] = 
 0
 0
 0
 0
[ CPUFloatType{4} ]
<<--->>



**写入或读取从字符串映射到张量的字典**

[注意]目前该功能仅存在于python对应的API中，因此libtorch暂时不支持；也有网友给出了解决方案，但是较为繁琐，此处不再深究；
https://github.com/pytorch/pytorch/issues/36568

In [6]:
// mydict = {'x': x, 'y': y}
// torch.save(mydict, 'mydict')
// mydict2 = torch.load('mydict')
// mydict2

**加载和保存模型参数**

In [7]:
/*design a net*/
struct MLP : torch::nn::Module {
  MLP() {
    // Construct and register two Linear submodules.
    hidden = register_module("hidden", torch::nn::Linear(20, 256));
    output = register_module("output", torch::nn::Linear(256, 10));
  }

  // Implement the Net's algorithm.
  torch::Tensor forward(torch::Tensor x) {
    // Use one of many tensor manipulation functions.
    x = hidden->forward(x);
    x = torch::relu(x);
    x = output->forward(x);
    return x;
  }

  // Use one of many "standard library" modules.
  torch::nn::Linear hidden{nullptr}, output{nullptr};
};

In [8]:
// Create a new Net.
auto net = std::make_shared<MLP>();
auto X = torch::randn({2, 20});
auto Y = net->forward(X);
printT(Y);

Y = 
 0.3339  0.1653 -0.3950 -0.1112 -0.2380 -0.0449  0.1673 -0.0204  0.2040 -0.1157
 0.2091  0.0271 -0.2066  0.3571  0.2958  0.2024  0.1747 -0.1816 -0.2225 -0.1623
[ CPUFloatType{2,10} ]
<<--->>



**将模型的参数存储为一个叫做“mlp.params”的文件**

In [9]:
torch::save(net, "mlp.params");

**实例化了原始多层感知机模型的一个备份。 直接读取文件中存储的参数**

In [10]:
// clone = MLP()
// clone.load_state_dict(torch.load("mlp.params"))
// clone.eval()
    
auto clone = std::make_shared<MLP>();
torch::load(clone, "mlp.params");
clone->eval();

In [11]:
// Y_clone = clone(X)
// Y_clone == Y

auto Y_clone = clone->forward(X);
if (Y_clone.equal(Y)) {
    std::cout << "Y_clone == Y" << std::endl;
}

Y_clone == Y
